## Linear Regression with TensorFlow and PyTorch

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

In [ ]:
import pickle
with open('mfr_data.pkl', 'rb') as handle:
    info = pickle.load(handle)
data,test,train,ds,s = info
data.head()

In [ ]:
# Split data/labels
data_X = np.array(data['H2R'])
data_Y = np.array(data['lnMFR'])

![exercise](https://apmonitor.com/che263/uploads/Begin_Python/exercise.png)

### Tensorflow Linear Regression

TensorFlow is known for Deep Learning. This tutorial shows how to use TensorFlow for a simple linear regression. There are better packages for linear regression. Also, Keras is an interface to TensorFlow that simplifies the model building, regression solution, and deployment.

In [ ]:
display_step = 100
learning_rate = 0.01
epochs = 1000

$$y=wx+b$$


In [ ]:
# Weight and Bias
w = tf.Variable(tf.zeros(1), name='weight')
b = tf.Variable(tf.zeros(1), name='bias')

def predict(x):
    return x * w + b

def mse(y_true, y_pred):
    return tf.losses.mean_squared_error(y_true,y_pred)

# SGD Optimizer
optimizer = tf.optimizers.SGD(learning_rate=learning_rate)

In [ ]:
history = []
start = time.time()

# Start training
for epoch in range(1, epochs + 1):
    
    # Begin GradientTape and optimise
    with tf.GradientTape() as g:
        pred = predict(data_X)
        loss = mse(data_Y, pred)

    # Compute dw, db
    gradients = g.gradient(loss, [w,b])
    
    # Update w and b
    optimizer.apply_gradients(zip(gradients, [w,b]))
    
    # Monitor training / display steps
    if epoch % display_step == 0:
        pred = predict(data_X)
        loss = mse(pred, data_Y)
        history.append(np.array(w * data_X + b))
        print("Epoch: %i, Loss: %f, w: %f, b: %f" % (epoch, loss, w.numpy(), b.numpy()))
print('Train Time: %f seconds' %(time.time() - start))

In [ ]:
fig = plt.figure(figsize=(10,6))

plt.scatter(data_X,data_Y, label="Data")
# for i in range(len(history)):
#     plt.plot(data_X,history[i], c='k',label=str((i+1)*display_step)+" epoch", alpha = (i+1)/(i+5))
plt.plot(data_X,history[-1],'k')
plt.xlabel('H2R')
plt.ylabel('lnMFR')
plt.show()

![exercise](https://apmonitor.com/che263/uploads/Begin_Python/exercise.png)

### Pytorch Linear Regression

PyTorch is best known for nonlinear regression such as with Deep Learning. This tutorial shows how to use PyTorch for a simple linear regression. There are better packages for linear regression. See also [Numpy vs PyTorch: Linear Regression from scratch](https://medium.com/analytics-vidhya/numpy-vs-pytorch-linear-regression-from-scratch-452a121fb0e8).

In [ ]:
data_X = data_X.reshape(len(data_X),1)
data_Y = data_Y.reshape(len(data_Y),1)

In [ ]:
import torch
from torch.autograd import Variable
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out

In [ ]:
inputDim = 1        # takes variable 'x' 
outputDim = 1       # takes variable 'y'
learningRate = 0.01 
epochs = 100

model = linearRegression(inputDim, outputDim)

In [ ]:
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

In [ ]:
for epoch in range(epochs):
    # Converting inputs and labels to Variable
    inputs = Variable(torch.from_numpy(data_X).float())
    labels = Variable(torch.from_numpy(data_Y).float())

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = model(inputs)

    # get loss for the predicted output
    loss = criterion(outputs, labels)
    print(loss)
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

#     print('epoch {}, loss {}'.format(epoch, loss.item()))

In [ ]:
with torch.no_grad(): # we don't need gradients in the testing phase
    predicted = model(Variable(torch.from_numpy(data_X).float())).data.numpy()
#     print(predicted)

fig = plt.figure(figsize=(10,6))
plt.scatter(data_X, data_Y)
plt.plot(data_X, predicted, 'k')
plt.xlabel('H2R')
plt.ylabel('lnMFR')
plt.show()